# Reinforcement Learning Model



In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install stable-baselines3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install "shimmy>=2.0"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gymnasium stable-baselines3 joblib numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# def train_default_agent():
#     """Train the PPO agent on the SecurityEnv using default parameters."""
#     # Create a unique run ID for the default run
#     from datetime import datetime
#     run_id = f"default_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
#     print(f"Starting training process with default parameters... (Run ID: {run_id})")

#     # Create directories with unique run ID
#     model_dir = f"models/run_{run_id}"
#     log_dir = f"logs/run_{run_id}"
    
#     os.makedirs(model_dir, exist_ok=True)
#     os.makedirs(log_dir, exist_ok=True)

#     # Create training and evaluation environments
#     train_env = create_env()
#     eval_env = create_env()

#     # Create the agent with default hyperparameters
#     model = PPO(
#         "MlpPolicy",
#         train_env,
#         # Using default values for all parameters
#         tensorboard_log=f"./{log_dir}/" if TENSORBOARD_AVAILABLE else None,
#         verbose=1  # Set to 1 to see training information
#     )

#     # Create callbacks
#     eval_callback = EvalCallback(
#         eval_env,
#         best_model_save_path=f"{model_dir}/best_model",
#         log_path=log_dir,
#         eval_freq=1000,
#         deterministic=True,
#         render=False
#     )

#     checkpoint_callback = CheckpointCallback(
#         save_freq=5000,
#         save_path=model_dir,
#         name_prefix="default_security_model"
#     )

#     # Train the agent
#     total_timesteps = 1000
#     model.learn(
#         total_timesteps=total_timesteps,
#         callback=[eval_callback, checkpoint_callback],
#         progress_bar=True
#     )

#     # Save the final model and normalization stats
#     model.save(f"{model_dir}/final_security_model")
#     train_env.save(f"{model_dir}/vec_normalize.pkl")

#     print(f"\nDefault training completed for run {run_id}!")
#     return model, train_env, run_id

# if __name__ == "__main__":
#     # Train the agent with optimized parameters
#     model, env, run_id = train_default_agent()

#     # Evaluate the agent using the raw environment 
#     eval_env = create_eval_env(render_mode="human")
#     evaluate_agent(model, eval_env)

In [7]:
import os
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
import matplotlib.pyplot as plt
from security_env import SecurityEnv

# Check if tensorboard is installed
try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_AVAILABLE = True
except ImportError:
    TENSORBOARD_AVAILABLE = False
    print("Tensorboard is not installed. Training logs will not be saved.")
    print("To install tensorboard, run: pip install tensorboard")

def create_env(render_mode=None):
    """Create and wrap the environment."""
    env = SecurityEnv(render_mode=render_mode)
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def train_agent():
    """Train the PPO agent on the SecurityEnv."""
    print("Starting training process...")
    
    # Create a unique run ID for the default run
    from datetime import datetime
    run_id = f"default_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print(f"Starting training process with default parameters... (Run ID: {run_id})")

    # Create directories with unique run ID
    model_dir = f"models/run_{run_id}"
    log_dir = f"logs/run_{run_id}"
    
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)

    # # Create directories for saving models and logs
    # os.makedirs("models", exist_ok=True)
    # os.makedirs("logs", exist_ok=True)

    # Create training and evaluation environments
    train_env = create_env()
    eval_env = create_env()

    # # Create the agent with optimized hyperparameters
    # model = PPO(
    #     "MlpPolicy",
    #     train_env,
    #     learning_rate=0.0003,
    #     n_steps=2048,
    #     batch_size=64,
    #     n_epochs=10,
    #     gamma=0.99,
    #     gae_lambda=0.95,
    #     clip_range=0.2,
    #     clip_range_vf=None,
    #     ent_coef=0.00,
    #     vf_coef=0.5,
    #     max_grad_norm=0.5,
    #     tensorboard_log="./logs/" if TENSORBOARD_AVAILABLE else None,
    #     policy_kwargs=dict(
    #         net_arch=dict(
    #             pi=[64, 64],
    #             vf=[64, 64]
    #         )
    #     ),
    #     verbose=0
    # )

        # Create the agent with default hyperparameters
    model = PPO(
        "MlpPolicy",
        train_env,
        # Using default values for all parameters
        tensorboard_log=f"./{log_dir}/" if TENSORBOARD_AVAILABLE else None,
        verbose=1  # Set to 1 to see training information
    )

    # Create callbacks
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path="./models/best_model",
        log_path="./logs",
        eval_freq=1000,
        deterministic=True,
        render=False
    )

    checkpoint_callback = CheckpointCallback(
        save_freq=5000,
        save_path="./models/",
        name_prefix="security_model"
    )

    # Train the agent
    total_timesteps = 1000  # Adjust based on your needs
    model.learn(
        total_timesteps=total_timesteps,
        callback=[eval_callback, checkpoint_callback],
        progress_bar=True
    )

    # Save the final model and normalization stats
    model.save("models/final_security_model")
    train_env.save("models/vec_normalize.pkl")

    print("\nTraining completed!")
    return model, train_env

def evaluate_agent(model, env, num_episodes=5):
    """Evaluate the trained agent."""
    print("\nEvaluating agent...")

    for episode in range(num_episodes):
        obs, _ = env.reset()
        done = False
        truncated = False
        total_reward = 0
        episode_steps = 0

        while not (done or truncated):
            action, _ = model.predict(obs, deterministic=True)

            result = env.step(action)
            if len(result) == 4:
                obs, reward, done, info = result
                truncated = False
            else:
                obs, reward, done, truncated, info = result

            if isinstance(reward, np.ndarray):
                reward = reward.item()

            total_reward += reward
            episode_steps += 1

            # Debugging: Check the type of info
            print(f"Type of info: {type(info)}")
            print(f"Info content: {info}")

            # Print step information
            print(f"\nEpisode {episode + 1}, Step {episode_steps}")
            print(f"Action: {action}")
            print(f"Reward: {reward:.2f}")
            print(f"Security Score: {info['security_score']:.2f}")
            print(f"Fatigue Score: {info['fatigue_score']:.2f}")
            print("Feature Values:")
            for name, value in info['feature_values'].items():
                print(f"  {name}: {value}")

        print(f"\nEpisode {episode + 1} completed:")
        print(f"Total Steps: {episode_steps}")
        print(f"Total Reward: {total_reward:.2f}")
        print("-" * 50) 

def plot_training_results(log_dir="./logs"):
    """Plot the training results."""
    import pandas as pd

    monitor_file = os.path.join(log_dir, "monitor.csv")
    if not os.path.exists(monitor_file):
        print(f"No log file found at {monitor_file}. Skipping plotting.")
        return

    # Read the training logs
    df = pd.read_csv(monitor_file, skiprows=1)

    # Plot episode rewards
    plt.figure(figsize=(12, 5))
    plt.plot(df['r'], label="Episode Reward")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title("Learning Curve")
    plt.legend()
    plt.savefig("logs/learning_curve.png")
    plt.close()

    # Plot moving average reward
    window_size = 10
    moving_avg = df['r'].rolling(window=window_size).mean()

    plt.figure(figsize=(12, 5))
    plt.plot(moving_avg, label=f"Moving Average Reward (window={window_size})", color='orange')
    plt.xlabel("Episode")
    plt.ylabel("Average Reward")
    plt.title("Moving Average of Episode Rewards")
    plt.legend()
    plt.savefig("logs/moving_average.png")
    plt.close()


def create_eval_env(render_mode=None):
    """Create a raw Gymnasium environment for evaluation."""
    env = SecurityEnv(render_mode=render_mode)
    return env

if __name__ == "__main__":
    # Train the agent
    model, env = train_agent()

    # Evaluate the agent using the raw environment to force Gymnasium API
    eval_env = create_eval_env(render_mode="human")
    evaluate_agent(model, eval_env)

    # Plot training results
    plot_training_results()

Starting training process...
Starting training process with default parameters... (Run ID: default_20250422_163608)
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'D

Output()

Eval num_timesteps=1000, episode_reward=332.40 +/- 73.71

Episode length: 100.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 100      |
|    mean_reward     | 332      |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------


...

Eval num_timesteps=2000, episode_reward=288.47 +/- 57.75

Episode length: 100.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 100      |
|    mean_reward     | 288      |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | 426      |
| time/              |          |
|    fps             | 69       |
|    iterations      | 1        |
|    time_elapsed    | 29       |
|    total_timesteps | 2048     |
---------------------------------



Training completed!
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']

Evaluating agent...
Type of info: <class 'dict'>
Info content: {'security_score': 18.4, 'fatigue_score': 1.7422222222222223, 'step': 1, 'feature_values': {'Familarity': 0.0, 'Frequency of Password Changes': 2.0, 'Difficulty Level Password': 3.0, 'Effort Required Password': 4.0, 'Perceived Importance